<a href="https://colab.research.google.com/github/cosmicishan/Classification-on-text-data/blob/main/73_strings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries and Dataset**




In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from gensim.models import Word2Vec

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/73-Strings/Train.csv')

In [ ]:
X = df.iloc[:,1].values
y = df.iloc[:,-1].values

# **Text Preprocessing**

In [ ]:
#Removing contraction from the text.

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase 

In [ ]:
#We are removing the words from the stop words list: 'no', 'nor', 'not' 
#we are including them into stop words list

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
from tqdm import tqdm
preprocessed_description = []
# tqdm is for printing the status bar
for sentance in tqdm(X):
    sentance = str(sentance)
    sentance = re.sub(r"http\S+", "", sentance) #Remove links
    sentance = BeautifulSoup(sentance, 'lxml').get_text() #Remove tags
    sentance = decontracted(sentance) #Remove Contraction
    sentance = re.sub("\S*\d\S*", "", sentance).strip() #Remove words with numbers
    sentance = re.sub('[^A-Za-z]+', ' ', sentance) #Remove Special Character
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_description.append(sentance.strip())

100%|██████████| 6045/6045 [00:02<00:00, 2713.17it/s]


# **Featurization**

## *Bag of Words*


In [ ]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_description)
print("some feature names : ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_description)
print("the type of count vectorizer : ",type(final_counts))
print("the shape of out text BOW vectorizer : ",final_counts.get_shape())
print("the number of unique words : ", final_counts.get_shape()[1])

some feature names :  ['aaa', 'aad', 'aadc', 'aaftink', 'aagp', 'aagps', 'aallon', 'aam', 'aaon', 'aap']
the type of count vectorizer :  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer :  (6045, 34494)
the number of unique words :  34494


In [ ]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(final_counts, y, test_size = 0.25)

**Training bag of word model on SVM algorithm**

In [ ]:
classifier = SVC(kernel = 'sigmoid')
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

bow_svm = accuracy_score(y_test,y_pred)

print("Accuracy of bag of words model in SVM alorithm : ",bow_svm)

Accuracy of bag of words model in SVM alorithm :  0.6223544973544973


**Training bag of word model on Random Forest algorithm**

In [ ]:
estimators = list(range(1,20))
cv_score = []

for n in estimators:
    random_forest = RandomForestClassifier(n_estimators = n, criterion = 'entropy')
    scores = cross_val_score(random_forest, X_train, y_train, cv=10, scoring='accuracy')
    cv_score.append(scores.mean())

bow_rf = max(cv_score)

print("Accuracy of bag of words model in Random Forest algorithm : ",bow_rf)

Accuracy of bag of words model in Random Forest algorithm :  0.40370899825927975


**Training bag of word model on Naive Bayes algorithm**

In [ ]:
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

classifier = GaussianNB()
classifier.fit(X_train_dense, y_train)

y_pred = classifier.predict(X_test_dense)

bow_nb = accuracy_score(y_test,y_pred)

print("Accuracy of bag of words model in Naive Bayes algorithm : ",bow_nb)

Accuracy of bag of words model in Naive Bayes algorithm :  0.5205026455026455


##*Bi-Grams and n-Grams*

In [ ]:
#bi-gram, tri-gram and n-gram

count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_description)
print("the type of count vectorizer : ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer : ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams : ", final_bigram_counts.get_shape()[1])

the type of count vectorizer :  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer :  (6045, 5000)
the number of unique words including both unigrams and bigrams :  5000


In [ ]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(final_bigram_counts, y, test_size = 0.25)

**Training unigrams/bigrams model on SVM algorithm**

In [ ]:
classifier = SVC(kernel = 'sigmoid')
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

grams_svm = accuracy_score(y_test,y_pred)

print("Accuracy of unigrams/bigrams model in SVM algorithm : ",grams_svm)

Accuracy of unigrams/bigrams model in SVM algorithm :  0.623015873015873


**Training unigrams/bigrams model on Random Forest algorithm**

In [ ]:
estimators = list(range(1,20))
cv_score = []

for n in estimators:
    random_forest = RandomForestClassifier(n_estimators = n, criterion = 'entropy')
    scores = cross_val_score(random_forest, X_train, y_train, cv=10, scoring='accuracy')
    cv_score.append(scores.mean())

grams_rf = max(cv_score)

print("Accuracy of unigrams/bigrams model in Random Forest algorithm : ",grams_rf)

Accuracy of unigrams/bigrams model in Random Forest algorithm :  0.4136340208691931


**Training unigrams/bigrams model on Naive Bayes algorithm**

In [ ]:
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

classifier = GaussianNB()
classifier.fit(X_train_dense, y_train)

y_pred = classifier.predict(X_test_dense)

grams_nb = accuracy_score(y_test,y_pred)

print("Accuracy of unigrams/bigrams model in Naive Bayes algorithm : ",grams_nb)

Accuracy of unigrams/bigrams model in Naive Bayes algorithm :  0.5205026455026455


## *TF_IDF*


In [ ]:
#tf-idf

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_description)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_description)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['ab', 'ab formerly', 'ab publ', 'ab sweden', 'ability', 'abitibi', 'abitibi greenstone', 'able', 'abroad', 'absorption']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (6045, 7682)
the number of unique words including both unigrams and bigrams  7682


In [ ]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(final_tf_idf, y, test_size = 0.25)

**Training TF_IDF model on SVM algorithm**

In [ ]:
classifier = SVC(kernel = 'sigmoid')
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

idf_svm = accuracy_score(y_test,y_pred)

print("Accuracy of TF-IDF model in SVM algorithm : ",idf_svm)

Accuracy of TF-IDF model in SVM algorithm :  0.6752645502645502


**Training TF_IDF model on Random Forest algorithm**

In [ ]:
estimators = list(range(1,20))
cv_score = []

for n in estimators:
    random_forest = RandomForestClassifier(n_estimators = n, criterion = 'entropy')
    scores = cross_val_score(random_forest, X_train, y_train, cv=10, scoring='accuracy')
    cv_score.append(scores.mean())

idf_rf = max(cv_score)

print("Accuracy of TF_IDF model in Random Forest algorithm : ",idf_rf)

Accuracy of TF_IDF model in Random Forest algorithm :  0.4103290836420923


**Training TF_IDF model on Naive Bayes algorithm**

In [ ]:
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

classifier = GaussianNB()
classifier.fit(X_train_dense, y_train)

y_pred = classifier.predict(X_test_dense)

idf_nb = accuracy_score(y_test,y_pred)

print("Accuracy of TF_IDF model in Naive Bayes algorithm : ",idf_nb)

Accuracy of TF_IDF model in Naive Bayes algorithm :  0.5337301587301587


## *Word2Vec*

In [ ]:
#Training Word2Vec using our own Text corpus.

list_of_sentance=[]
for sentance in preprocessed_description:
    list_of_sentance.append(sentance.split())

w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
print("Words similar to Company according to our corpus : ",w2v_model.wv.most_similar('company'))
print('='*50)
print("Words similar to Technologies according to our corpus : ",w2v_model.wv.most_similar('technologies'))

w2v_words = list(w2v_model.wv.vocab)
print('='*50)
print("Number of words that occured minimum 5 times : ",len(w2v_words))
print("Sample words ", w2v_words[0:50])

Words similar to Company according to our corpus :  [('acquiring', 0.6578993797302246), ('principally', 0.6536482572555542), ('launching', 0.6526538729667664), ('licensing', 0.6384079456329346), ('owning', 0.6356709599494934), ('empowered', 0.6335441470146179), ('operating', 0.6292605400085449), ('diversified', 0.628777265548706), ('provision', 0.6278267502784729), ('cannabis', 0.6204374432563782)]
Words similar to Technologies according to our corpus :  [('diagnostics', 0.9511491656303406), ('advanced', 0.891674816608429), ('science', 0.8850345015525818), ('pharmaceuticals', 0.8765751719474792), ('cgmp', 0.8624785542488098), ('biomedical', 0.8619247078895569), ('laboratory', 0.860388457775116), ('sciences', 0.8592849969863892), ('semiconductor', 0.8575431108474731), ('technology', 0.8572807908058167)]
Number of words that occured minimum 5 times :  8311
Sample words  ['partners', 'inc', 'provides', 'advertising', 'agency', 'services', 'specializing', 'direct', 'response', 'media', 'ca

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## *Converting text into vectors using Avg W2V and TFIDF-W2V*

### *Average Word2Vec*

In [ ]:
# average Word2Vec
# compute average word2vec for each description.
sent_vectors = []; # the avg-w2v for each sentence/description is stored in this list
for sent in tqdm(list_of_sentance): # for each description/sentence
    sent_vec = np.zeros(50) 
    cnt_words =0; # num of words with a valid vector in the sentence/description
    for word in sent: # for each word in a description/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)

print(len(sent_vectors))

100%|██████████| 6045/6045 [00:21<00:00, 275.83it/s]

6045


In [ ]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(sent_vectors, y, test_size = 0.25)

**Training Average Word2Vec model on SVM algorithm**

In [ ]:
classifier = SVC(kernel = 'sigmoid')
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

w2v_svm = accuracy_score(y_test,y_pred)

print("Accuracy of Word2Vec model in SVM algorithm : ",w2v_svm)

Accuracy of Word2Vec model in SVM algorithm :  0.35714285714285715


**Training Average Word2Vec model on Random Forest algorithm**

In [ ]:
estimators = list(range(1,20))
cv_score = []

for n in estimators:
    random_forest = RandomForestClassifier(n_estimators = n, criterion = 'entropy')
    scores = cross_val_score(random_forest, X_train, y_train, cv=10, scoring='accuracy')
    cv_score.append(scores.mean())

w2v_rf = max(cv_score)

print("Accuracy of Word2Vec model in Random Forest algorithm : ",w2v_rf)

Accuracy of Word2Vec model in Random Forest algorithm :  0.4502513833377095


**Training Average Word2Vec model on Naive Bayes**

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

w2v_nb = accuracy_score(y_test,y_pred)

print("Accuracy of Word2Vec model in Naive Bayes algorithm : ",w2v_nb)

Accuracy of Word2Vec model in Naive Bayes algorithm :  0.4279100529100529


### *TF-IDF Word2Vec*

In [ ]:
model = TfidfVectorizer()
model.fit(preprocessed_description)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/description is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each description/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/description
    for word in sent: # for each word in a description/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|██████████| 6045/6045 [02:53<00:00, 34.94it/s]


In [ ]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(tfidf_sent_vectors, y, test_size = 0.25)

**Training TF-IDF Word2Vec model on SVM**

In [ ]:
classifier = SVC(kernel = 'sigmoid')
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

tfw2v_svm = accuracy_score(y_test,y_pred)

print("Accuracy of TF-IDF Word2Vec model in SVM algorithm : ",tfw2v_svm)

Accuracy of TF-IDF Word2Vec model in SVM algorithm :  0.30886243386243384


**Training TF-IDF Word2Vec model on Random Forest**

In [ ]:
estimators = list(range(1,20))
cv_score = []

for n in estimators:
    random_forest = RandomForestClassifier(n_estimators = n, criterion = 'entropy')
    scores = cross_val_score(random_forest, X_train, y_train, cv=10, scoring='accuracy')
    cv_score.append(scores.mean())

tfw2v_rf = max(cv_score)

print("Accuracy of TF_IDF Word2Vec model in Random Forest algorithm : ",tfw2v_rf)

Accuracy of TF_IDF Word2Vec model in Random Forest algorithm :  0.45687244118991355


**Training TF-IDF Word2Vec model on Naive Bayes**

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

tfw2v_nb = accuracy_score(y_test,y_pred)

print("Accuracy of TF_IDF Word2Vec model in Naive Bayes algorithm : ",tfw2v_nb)

Accuracy of TF_IDF Word2Vec model in Naive Bayes algorithm :  0.38955026455026454
